##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 이미지 분류를 위한 Federated Learning

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/federated/tutorials/federated_learning_for_image_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/federated/tutorials/federated_learning_for_image_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소그 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/federated/tutorials/federated_learning_for_image_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

**참고**: 이 Colab은 `tensorflow_federated` pip 패키지의 [최신 릴리즈 버전](https://github.com/tensorflow/federated#compatibility)에서 동작하는 것으로 확인되었지만 Tensorflow Federated 프로젝트는 아직 시험판 개발 중이며 `main`에서 동작하지 않을 수 있습니다.

이 튜토리얼에서는 고전적인 MNIST 훈련 예제를 사용하여 TFF의 Federated Learning(FL) API 레이어(`tff.learning` - TensorFlow에서 구현된 사용자 제공 모델에 대해 페더레이션 훈련과 같은 일반적인 유형의 페더레이션 학습 작업을 수행하는 데 사용할 수 있는 상위 수준의 인터페이스 세트)를 소개합니다.

이 튜토리얼과 Federated Learning API는 주로 자신의 TensorFlow 모델을 TFF에 연결하여 후자를 대부분 블랙 박스로 취급하려는 사용자를 대상으로 합니다. TFF에 대한 심층적인 이해와 자신의 페더레이션 학습 알고리즘을 구현하는 방법은 FC Core API 튜토리얼 - [사용자 정의 페더레이션 알고리즘 1부](custom_federated_algorithms_1.ipynb) 및 [2부](custom_federated_algorithms_2.ipynb)를 참조하세요.

<code>tff.learning</code>에 대한 자세한 내용은 <a>Text Generation용 Federated Learning</a> 튜토리얼에서 계속하세요. 반복 모델을 다루는 것 외에도 Keras를 사용한 평가와 결합된 페더레이션 학습을 통해 구체화를 위한 사전 훈련되고 직렬화된 Keras 모델을 로드하는 방법을 보여줍니다.

## 시작하기 전에

시작하기 전에 다음을 실행하여 환경이 올바르게 설정되었는지 확인합니다. 인사말이 표시되지 않으면 [설치](../install.md) 가이드에서 지침을 참조하세요. 

In [ ]:
#@test {"skip": true}

!pip install --quiet --upgrade tensorflow-federated

In [ ]:
%load_ext tensorboard

Fetching TensorBoard MPM version 'live'... done.


In [ ]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

## 입력 데이터 준비하기

데이터부터 시작하겠습니다. 페더레이션 학습에는 페더레이션 데이터세트, 즉 여러 사용자의 데이터 모음이 필요합니다. 페더레이션 데이터는 일반적으로 고유한 문제를 제기하는 비 [i.i.d.](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables)입니다.

실험을 용이하게 하기 위해 [Leaf](https://www.nist.gov/srd/nist-special-database-19)를 사용하여 재처리된 [원래 NIST 데이터세트](https://github.com/TalwalkarLab/leaf)의 버전이 포함된 MNIST의 페더레이션 버전을 포함하여 몇 개의 데이터세트로 TFF 리포지토리를 시드하여 데이터가 원래 숫자 작성자에 의해 입력되도록 합니다. 작성자마다 고유한 스타일이 있기 때문에 이 데이터세트는 페더레이션 데이터세트에서 예상되는 non-i.i.d. 동작의 종류를 보여줍니다.

로드하는 방법은 다음과 같습니다.

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

`load_data()`가 반환하는 데이터세트는 사용자 세트를 열거하고 특정 사용자의 데이터를 나타내는 `tf.data.Dataset`를 구성하고 개별 요소의 구조를 쿼리할 수 있는 인터페이스인 `tff.simulation.ClientData`의 인스턴스입니다. 이 인터페이스를 사용하여 데이터세트의 내용을 탐색하는 방법은 다음과 같습니다. 이 인터페이스를 사용하면 클라이언트 ID를 반복할 수 있지만, 이는 시뮬레이션 데이터의 특성일 뿐입니다. 곧 살펴보겠지만, 클라이언트 ID는 페더레이션 학습 프레임워크에서 사용되지 않습니다. 클라이언트 ID의 유일한 목적은 시뮬레이션을 위해 데이터의 하위 집합을 선택할 수 있도록 하는 것입니다.

In [ ]:
len(emnist_train.client_ids)

3383

In [ ]:
emnist_train.element_type_structure

OrderedDict([('label', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('pixels', TensorSpec(shape=(28, 28), dtype=tf.float32, name=None))])

In [ ]:
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])

example_element = next(iter(example_dataset))

example_element['label'].numpy()

1

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(example_element['pixels'].numpy(), cmap='gray', aspect='equal')
plt.grid(False)
_ = plt.show()

### 페더레이션 데이터의 이질성 탐색하기

페더레이션 데이터는 일반적으로 비 [i.i.d.](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables)이며, 사용자는 일반적으로 사용 패턴에 따라 데이터 분포가 다릅니다. 일부 클라이언트는 기기에 훈련 예제가 적어 로컬에서 데이터가 부족할 수 있지만, 일부 클라이언트는 충분한 훈련 예제를 가지고 있습니다. 사용 가능한 EMNIST 데이터를 사용하여 페더레이션 시스템의 일반적인 데이터 이질성 개념을 살펴보겠습니다. 고객 데이터에 대한 이 심층 분석은 모든 데이터를 로컬에서 사용할 수 있는 시뮬레이션 환경이기 때문에 당사만 사용할 수 있다는 점에 유의하는 것이 중요합니다. 실제 운영 페더레이션 환경에서는 단일 클라이언트의 데이터를 검사할 수 없습니다.

먼저, 하나의 시뮬레이션 기기에서 예제에 대한 느낌을 얻기 위해 한 클라이언트의 데이터를 샘플링해 보겠습니다. 당사가 사용하는 데이터세트는 고유한 작성자가 입력했기 때문에 한 클라이언트의 데이터는 한 사용자의 고유한 "사용 패턴"을 시뮬레이션하여 0부터 9까지의 숫자 샘플에 대한 한 사람의 손글씨를 나타냅니다.

In [ ]:
## Example MNIST digits for one client
figure = plt.figure(figsize=(20, 4))
j = 0

for example in example_dataset.take(40):
  plt.subplot(4, 10, j+1)
  plt.imshow(example['pixels'].numpy(), cmap='gray', aspect='equal')
  plt.axis('off')
  j += 1

이제 각 MNIST 숫자 레이블에 대한 각 클라이언트의 예제 수를 시각화해 보겠습니다. 페더레이션 환경에서 각 클라이언트의 예제 수는 사용자 동작에 따라 상당히 다를 수 있습니다.

In [ ]:
# Number of examples per layer for a sample of clients
f = plt.figure(figsize=(12, 7))
f.suptitle('Label Counts for a Sample of Clients')
for i in range(6):
  client_dataset = emnist_train.create_tf_dataset_for_client(
      emnist_train.client_ids[i])
  plot_data = collections.defaultdict(list)
  for example in client_dataset:
    # Append counts individually per label to make plots
    # more colorful instead of one color per plot.
    label = example['label'].numpy()
    plot_data[label].append(label)
  plt.subplot(2, 3, i+1)
  plt.title('Client {}'.format(i))
  for j in range(10):
    plt.hist(
        plot_data[j],
        density=False,
        bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

이제 각 MNIST 레이블에 대한 클라이언트별 평균 이미지를 시각화해 보겠습니다. 이 코드는 하나의 레이블에 대한 사용자의 모든 예제에 대한 각 픽셀 값의 평균을 생성합니다. 한 고객의 숫자에 대한 평균 이미지는 각 개인의 고유한 필기 스타일로 인해 같은 숫자에 대한 다른 고객의 평균 이미지와 다르게 보일 것입니다. 해당 로컬 라운드에서 해당 사용자의 고유 한 데이터에서 학습하므로 각 로컬 훈련 라운드가 각 클라이언트에서 다른 방향으로 모델을 어떻게 움직일지 뮤즈할 수 있습니다. 튜토리얼의 뒷부분에서 모든 클라이언트의 모델에 대한 각 업데이트를 가져와서 각 클라이언트의 고유한 데이터에서 학습한 새로운 글로벌 모델로 통합하는 방법을 살펴보겠습니다.

In [ ]:
# Each client has different mean images, meaning each client will be nudging
# the model in their own directions locally.

for i in range(5):
  client_dataset = emnist_train.create_tf_dataset_for_client(
      emnist_train.client_ids[i])
  plot_data = collections.defaultdict(list)
  for example in client_dataset:
    plot_data[example['label'].numpy()].append(example['pixels'].numpy())
  f = plt.figure(i, figsize=(12, 5))
  f.suptitle("Client #{}'s Mean Image Per Label".format(i))
  for j in range(10):
    mean_img = np.mean(plot_data[j], 0)
    plt.subplot(2, 5, j+1)
    plt.imshow(mean_img.reshape((28, 28)))
    plt.axis('off')

사용자 데이터는 노이즈가 많고 레이블이 안정적이지 않을 수 있습니다. 예를 들어, 위의 클라이언트 #2의 데이터를 살펴보면 레이블 2의 경우, 노이즈가 더 많은 평균 이미지를 생성하는 레이블이 잘못 지정된 예가 있을 수 있습니다.

### 입력 데이터 전처리

데이터가 이미 `tf.data.Dataset`이므로 데이터세트 변환을 사용하여 전처리를 수행할 수 있습니다. 여기에서는 `28x28` 이미지를 `784`개 요소 배열로 병합하고, 개별 예를 셔플하고, 배치로 구성하고, Keras와 함께 사용할 수 있도록 특성의 이름을 `pixels` 및 `label`에서 `x` 및 `y`로 바꿉니다. 또한, 데이터세트를 `repeat`하여 여러 epoch를 실행합니다.

In [ ]:
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 784]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

이것이 작동하는지 확인해 보겠습니다.

In [ ]:
preprocessed_example_dataset = preprocess(example_dataset)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))

sample_batch

OrderedDict([('x', array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)), ('y', array([[2],
       [1],
       [5],
       [7],
       [1],
       [7],
       [7],
       [1],
       [4],
       [7],
       [4],
       [2],
       [2],
       [5],
       [4],
       [1],
       [1],
       [0],
       [0],
       [9]], dtype=int32))])

당사는 페더레이션 데이터세트를 구성하기 위한 거의 모든 구성 요소를 갖추고 있습니다.

시뮬레이션에서 페더레이션 데이터를 TFF에 공급하는 방법 중 하나는 목록의 각 요소가 목록이든 `tf.data.Dataset`이든 상관없이 개별 사용자의 데이터를 보유하는 목록의 각 요소를 사용하여 간단히 Python 목록으로 만드는 것입니다. 후자를 제공하는 인터페이스가 이미 있으므로 사용해 보겠습니다.

다음은 훈련 또는 평가 라운드에 대한 입력으로 주어진 사용자 세트의 데이터세트 목록을 구성하는 간단한 도우미 함수입니다.

In [ ]:
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

이제 클라이언트를 어떻게 선택할까요?

일반적인 페더레이션 훈련 시나리오에서는 잠재적으로 매우 많은 수의 사용자 기기를 다루고 있으며, 이 중 일부만 주어진 시점에서 훈련에 사용할 수 있습니다. 예를 들어, 클라이언트 기기가 전원에 연결되어 있고 데이터 통신 연결 네트워크가 꺼져 있거나 유휴 상태일 때만 훈련에 참여하는 휴대폰인 경우입니다.

물론, 시뮬레이션 환경에서는 모든 데이터를 로컬에서 사용할 수 있습니다. 통상적으로, 시뮬레이션을 실행할 때 일반적으로 각 라운드마다 다른 각 훈련 라운드에 참여할 클라이언트의 무작위 하위 집합을 샘플링합니다.

즉, [Federated Averaging](https://arxiv.org/abs/1602.05629) 알고리즘에 대한 논문을 연구하면 알 수 있듯이, 각 라운드에 무작위로 샘플링된 클라이언트 하위 집합이 있는 시스템에서 수렴을 달성하는 데는 시간이 걸릴 수 있으며, 이 대화형 튜토리얼에서 수백 번의 라운드를 실행해야 하는 것은 비현실적입니다.

대신 클라이언트 세트를 한 번 샘플링하고 수렴 속도를 높이기 위해 라운드에서 같은 세트를 재사용할 것입니다(의도적으로 이들 소수의 사용자 데이터에 과대적합임). 독자가 이 튜토리얼을 수정하여 무작위 샘플링을 시뮬레이션하는 것은 연습으로 남겨 둡니다. 매우 쉽습니다(한 번 수행하면 모델을 수렴하는 데 시간이 걸릴 수 있음을 명심하세요).

In [ ]:
sample_clients = emnist_train.client_ids[0:NUM_CLIENTS]

federated_train_data = make_federated_data(emnist_train, sample_clients)

print(f'Number of client datasets: {len(federated_train_data)}')
print(f'First dataset: {federated_train_data[0]}')

Number of client datasets: 10
First dataset: <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))])>


## Keras로 모델 만들기

Keras를 사용하는 경우, Keras 모델을 구성하는 코드가 이미 있을 수 있습니다. 다음은 요구 사항에 맞는 간단한 모델의 예제입니다.

In [ ]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])

**참고:** 아직 모델을 컴파일하지 않습니다. 손실, 메트릭 및 옵티마이저는 나중에 소개됩니다.

TFF와 함께 모델을 사용하려면 `tff.learning.models.VariableModel` 인터페이스의 인스턴스로 래핑해야 합니다. 이 인터페이스는 Keras와 유사하게 모델의 순방향 전달, 메타데이터 속성 등을 스탬프 처리하는 메서드를 노출하지만 페더레이션 메트릭 계산 프로세스를 제어하는 방법과 같은 추가 요소도 도입합니다. 지금 당장은 이러한 내용을 걱정하지 않아도 됩니다. 위에서 방금 정의한 것과 같은 Keras 모델이 있는 경우, 아래와 같이 `tff.learning.models.from_keras_model`을 호출하여 모델과 샘플 데이터 배치를 인수로 전달하여 TFF가 모델을 래핑하도록 할 수 있습니다.

In [ ]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## 페더레이션 데이터로 모델 훈련하기

이제 TFF와 함께 사용하기 위해 `tff.learning.models.VariableModel`로 래핑된 모델이 준비되었으므로 다음과 같이 도우미 함수 `tff.learning.algorithms.build_weighted_fed_avg`를 호출하여 TFF가 Federated Averaging 알고리즘을 구성하도록 할 수 있습니다.

인수는 이미 생성된 인스턴스가 아닌 생성자(예: 위의 `model_fn`)여야 하므로 모델 생성은 TFF에 의해 제어되는 컨텍스트에서 발생할 수 있습니다(그 이유가 궁금하다면, [사용자 정의 알고리즘](custom_federated_algorithms_1.ipynb)에 대한 후속 튜토리얼을 읽어 보시기 바랍니다).

아래의 Federated Averaging 알고리즘에 대한 중요한 참고 사항 중 하나는 *client_optimizer* 및 *server_optimizer의* **두 가지** 옵티마이저입니다. *client_optimizer*는 각 클라이언트에서 로컬 모델 업데이트를 계산하는 데만 사용됩니다. *server_optimizer*는 평균 업데이트를 서버의 글로벌 모델에 적용합니다. 특히, 이는 사용되는 옵티마이저 및 학습률의 선택이 표준 iid 데이터세트에 대해 모델을 훈련하는 데 사용한 것과 달라야 할 수 있음을 의미합니다. 정규 SGD부터 시작하는 것이 좋습니다. 학습률이 평소보다 낮을 수 있습니다. 여기서 사용하는 학습률은 신중하게 조정되지 않았으므로 자유롭게 실험해 보세요.

In [ ]:
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

방금 무슨 일이 있었나요? TFF는 한 쌍의 *페더레이션 계산*을 구성하고 `tff.templates.IterativeProcess`로 패키징하여 이들 계산을 한 쌍의 속성 `initialize` 및 `next`로 사용할 수 있습니다.

간단히 말해서, *페더레이션 계산*은 다양한 페더레이션 알고리즘을 표현할 수 있는 TFF의 내부 언어로 된 프로그램입니다([사용자 정의 알고리즘](custom_federated_algorithms_1.ipynb) 튜토리얼에서 자세한 내용을 찾을 수 있음). 이 경우, 생성되고 `iterative_process`로 패키징된 두 가지 계산은 [Federated Averaging](https://arxiv.org/abs/1602.05629)을 구현합니다.

실제 페더레이션 학습 설정에서 실행될 수 있는 방식으로 계산을 정의하는 것이 TFF의 목표이지만, 현재는 로컬 실행 시뮬레이션 런타임만 구현됩니다. 시뮬레이터에서 계산을 실행하려면 Python 함수처럼 간단히 호출하면 됩니다. 이 기본 해석 환경은 고성능을 위해 설계되지 않았지만, 이 튜토리얼에는 충분합니다. 향후 릴리스에서 대규모 연구를 용이하게 하기 위해 고성능 시뮬레이션 런타임을 제공할 것으로 기대합니다.

`initialize` 계산부터 시작하겠습니다. 모든 페더레이션 계산의 경우와 마찬가지로 이를 함수로 생각할 수 있습니다. 계산은 인수를 사용하지 않고 하나의 결과를 반환합니다. 즉, 서버에서 Federated Averaging 프로세스의 상태를 나타냅니다. TFF의 세부 사항에 대해 자세히 알아보고 싶지는 않지만, 이 상태가 어떻게 생겼는지 확인하는 것이 도움이 될 수 있습니다. 다음과 같이 시각화할 수 있습니다.

In [ ]:
print(training_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[784,10],
      float32[10]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64,
    float32[784,10],
    float32[10]
  >
>@SERVER)


위의 유형 서명이 처음에는 다소 모호해 보일 수 있지만 서버 상태가 `global_model_weights`(모든 장치에 배포될 MNIST의 초기 모델 매개변수), 일부 빈 매개변수(서버-클라이언트 통신을 제어하는 `distributor` 등) 및 `finalizer` 요소로 구성된다는 사실을 알 수 있습니다. 이 마지막 요소는 서버가 라운드 종료 시 모델을 업데이트하는 데 사용하는 논리를 제어하며 FedAvg 라운드 수를 나타내는 정수를 포함합니다.

`initialize` 계산을 호출하여 서버 상태를 구성해 보겠습니다.

In [ ]:
train_state = training_process.initialize()

두 번째 페더레이션 계산 쌍인 `next`는 서버 상태(모델 매개변수 포함)를 클라이언트에 푸시, 로컬 데이터에 대한 기기 내 훈련, 모델 업데이트 수집 및 평균화로 구성된 단일 라운드의 페더레이션 평균화를 나타내며, 서버에서 업데이트된 새 모델을 생성합니다.

개념적으로, `next`과 같은 함수형 형식 서명을 갖는 것으로 생각할 수 있습니다.

```
SERVER_STATE, FEDERATED_DATA -> SERVER_STATE, TRAINING_METRICS
```

특히, `next()`는 서버에서 실행되는 함수가 아니라 전체 분산 계산의 선언적 함수형 표현으로 생각해야 합니다. 일부 입력은 서버( `SERVER_STATE`)에서 제공하지만, 참여하는 각 기기는 자체 로컬 데이터트를 제공합니다.

훈련을 한 라운드 실행하고 결과를 시각화해 보겠습니다. 사용자의 샘플을 위해 위에서 이미 생성한 페더레이션 데이터를 사용할 수 있습니다.

In [ ]:
result = training_process.next(train_state, federated_train_data)
train_state = result.state
train_metrics = result.metrics
print('round  1, metrics={}'.format(train_metrics))

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.12345679), ('loss', 3.1193733), ('num_examples', 4860), ('num_batches', 248)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


몇 라운드 더 실행해 봅시다. 앞서 언급했듯이, 일반적으로 이 시점에서는 사용자가 지속적으로 오고가는 현실적인 배포를 시뮬레이션하기 위해 각 라운드에서 무작위로 선택한 새로운 사용자 샘플에서 시뮬레이션 데이터의 하위 세트를 선택합니다. 그러나 이 대화형 노트북에서는 데모를 위해 같은 사용자만 재사용하여 시스템이 빠르게 수렴되도록 합니다.

In [ ]:
NUM_ROUNDS = 11
for round_num in range(2, NUM_ROUNDS):
  result = training_process.next(train_state, federated_train_data)
  train_state = result.state
  train_metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, train_metrics))

round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.14012346), ('loss', 2.9851403), ('num_examples', 4860), ('num_batches', 248)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  3, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.1590535), ('loss', 2.8617127), ('num_examples', 4860), ('num_batches', 248)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  4, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('sparse_categorical_accuracy', 0.17860082), ('loss', 2.7401376), ('num_examples', 4860), ('num_batches', 248)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('fin

페더레이션 훈련의 각 라운드 후에 훈련 손실이 감소하여 모델이 수렴되고 있음을 나타냅니다. 이러한 훈련 메트릭에는 몇 가지 중요한 주의 사항이 있지만, 이 튜토리얼 뒷부분의 *평가* 섹션을 참조하세요.

## TensorBoard에 모델 메트릭 표시, 다음으로 Tensorboard를 사용하여 이들 페더레이션 계산의 메트릭을 시각화해 보겠습니다.

다음으로 Tensorboard를 사용하여 이들 페더레이션 계산의 메트릭을 시각화해 보겠습니다.

메트릭을 기록할 디렉터리와 해당 요약 작성기를 만드는 것으로 시작하겠습니다.


In [ ]:
#@test {"skip": true}
logdir = "/tmp/logs/scalars/training/"
try:
  tf.io.gfile.rmtree(logdir)  # delete any previous results
except tf.errors.NotFoundError as e:
  pass # Ignore if the directory didn't previously exist.
summary_writer = tf.summary.create_file_writer(logdir)
train_state = training_process.initialize()

같은 요약 작성기를 사용하여 관련 스칼라 메트릭을 플롯합니다.

In [ ]:
#@test {"skip": true}
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    result = training_process.next(train_state, federated_train_data)
    train_state = result.state
    train_metrics = result.metrics
    for name, value in train_metrics['client_work']['train'].items():
      tf.summary.scalar(name, value, step=round_num)

위에서 지정한 루트 로그 디렉터리로 TensorBoard를 시작합니다. 데이터를 로드하는 데 몇 초 정도 걸릴 수 있습니다.

In [ ]:
#@test {"skip": true}
!ls {logdir}
%tensorboard --logdir {logdir} --port=0

In [ ]:
#@test {"skip": true}
# Uncomment and run this cell to clean your directory of old output for
# future graphs from this directory. We don't run it by default so that if 
# you do a "Runtime > Run all" you don't lose your results.

# !rm -R /tmp/logs/scalars/*

같은 방식으로 평가 메트릭을 보려면 "logs/scalars/eval"과 같은 별도의 eval 폴더를 만들어 TensorBoard에 쓸 수 있습니다.

## 모델 구현 사용자 정의하기

Keras는 [TensorFlow에 권장되는 상위 수준 모델 API](https://medium.com/tensorflow/standardizing-on-keras-guidance-on-high-level-apis-in-tensorflow-2-0-bad2b04c819a)이며, 가능하면 TFF에서 Keras 모델(`tff.learning.models.from_keras_model`을 통해)을 사용하는 것이 좋습니다.

다만, `tff.learning`은 페더레이션 학습을 위해 모델을 사용하는 데 필요한 최소한의 기능을 노출하는 하위 수준 모델 인터페이스인 `tff.learning.models.VariableModel`을 제공합니다. 이 인터페이스를 직접 구현하면(`tf.keras.layers`와 같은 빌딩 블록을 사용할 수도 있음) 페더레이션 학습 알고리즘의 내부를 수정하지 않고도 최대한 사용자 정의할 수 있습니다.

처음부터 다시 한번 해봅시다.

### 모델 변수, 순방향 전달 및 메트릭 정의하기

첫 번째 단계는 작업할 TensorFlow 변수를 식별하는 것입니다. 다음 코드를 더 읽기 쉽게 만들기 위해 전체 집합을 나타내는 데이터 구조를 정의하겠습니다. 여기에는 훈련할 `weights`와 `bias`와 같은 변수와 함께 `loss_sum`, `accuracy_sum` 및 `num_examples`와 같은 훈련 중에 업데이트할 다양한 누적 통계 및 카운터를 보유하는 변수도 포함됩니다.

In [ ]:
MnistVariables = collections.namedtuple(
    'MnistVariables', 'weights bias num_examples loss_sum accuracy_sum')

다음은 변수를 생성하는 메서드입니다. 간단하게 하기 위해 모든 통계를 `tf.float32`로 표시합니다. 그러면 이후 단계에서 유형 변환이 필요하지 않습니다. 변수 이니셜라이저를 람다로 래핑하는 것은 [리소스 변수](https://www.tensorflow.org/api_docs/python/tf/enable_resource_variables)에서 요구하는 사항입니다.

In [ ]:
def create_mnist_variables():
  return MnistVariables(
      weights=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(784, 10)),
          name='weights',
          trainable=True),
      bias=tf.Variable(
          lambda: tf.zeros(dtype=tf.float32, shape=(10)),
          name='bias',
          trainable=True),
      num_examples=tf.Variable(0.0, name='num_examples', trainable=False),
      loss_sum=tf.Variable(0.0, name='loss_sum', trainable=False),
      accuracy_sum=tf.Variable(0.0, name='accuracy_sum', trainable=False))

모델 매개변수 및 누적 통계에 대한 변수를 사용하여 다음과 같이 손실을 계산하고, 예측값을 내보내고, 단일 배치의 입력 데이터에 대한 누적 통계를 업데이트하는 순방향 전달 메서드를 정의할 수 있습니다.

In [ ]:
def predict_on_batch(variables, x):
  return tf.nn.softmax(tf.matmul(x, variables.weights) + variables.bias)

def mnist_forward_pass(variables, batch):
  y = predict_on_batch(variables, batch['x'])
  predictions = tf.cast(tf.argmax(y, 1), tf.int32)

  flat_labels = tf.reshape(batch['y'], [-1])
  loss = -tf.reduce_mean(
      tf.reduce_sum(tf.one_hot(flat_labels, 10) * tf.math.log(y), axis=[1]))
  accuracy = tf.reduce_mean(
      tf.cast(tf.equal(predictions, flat_labels), tf.float32))

  num_examples = tf.cast(tf.size(batch['y']), tf.float32)

  variables.num_examples.assign_add(num_examples)
  variables.loss_sum.assign_add(loss * num_examples)
  variables.accuracy_sum.assign_add(accuracy * num_examples)

  return loss, predictions

다음은 다시 TensorFlow를 사용하여 로컬 메트릭과 관련된 두 가지 함수를 정의하는 것입니다.

첫 번째 함수 `get_local_unfinalized_metrics`는 페더레이션 훈련 또는 평가 프로세스에서 서버에 집계할 수 있는 미완료 메트릭 값(자동으로 처리되는 모델 업데이트에 추가)을 반환합니다.

In [ ]:
def get_local_unfinalized_metrics(variables):
  return collections.OrderedDict(
      num_examples=[variables.num_examples],
      loss=[variables.loss_sum, variables.num_examples],
      accuracy=[variables.accuracy_sum, variables.num_examples])

두 번째 함수 `get_metric_finalizers`는 `get_local_unfinalized_metrics`와 동일한 키(즉, 메트릭 이름)를 사용하여 `tf.function`의 `OrderedDict`를 반환합니다. 각 `tf.function`은 메트릭의 미완료 값을 가져와 최종 메트릭을 계산합니다.

In [ ]:
def get_metric_finalizers():
  return collections.OrderedDict(
      num_examples=tf.function(func=lambda x: x[0]),
      loss=tf.function(func=lambda x: x[0] / x[1]),
      accuracy=tf.function(func=lambda x: x[0] / x[1]))

페더레이션 훈련 또는 평가 프로세스를 정의할 경우 `get_local_unfinalized_metrics`로 반환된 로컬 미완료 메트릭이 클라이언트 전체에서 집계되는 방식은 `metrics_aggregator` 매개변수로 지정됩니다. 예를 들어 [`tff.learning.algorithms.build_weighted_fed_avg`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg) API(다음 섹션 참조)에서 `metrics_aggregator`의 기본값은 [`tff.learning.metrics.sum_then_finalize`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/metrics/sum_then_finalize)이며 이는 먼저 `CLIENTS`의 미완료 메트릭을 합산한 다음 `SERVER`에서 메트릭 종료자를 적용합니다.

### `tff.learning.models.VariableModel` 인스턴스 구성하기

위의 모든 항목이 준비되었으므로 TFF가 Keras 모델을 수집하도록 할 때 생성되는 것과 유사한 TFF와 함께 사용할 모델 표현을 구성할 준비가 되었습니다.

In [ ]:
import collections
from collections.abc import Callable

class MnistModel(tff.learning.models.VariableModel):

  def __init__(self):
    self._variables = create_mnist_variables()

  @property
  def trainable_variables(self):
    return [self._variables.weights, self._variables.bias]

  @property
  def non_trainable_variables(self):
    return []

  @property
  def local_variables(self):
    return [
        self._variables.num_examples, self._variables.loss_sum,
        self._variables.accuracy_sum
    ]

  @property
  def input_spec(self):
    return collections.OrderedDict(
        x=tf.TensorSpec([None, 784], tf.float32),
        y=tf.TensorSpec([None, 1], tf.int32))

  @tf.function
  def predict_on_batch(self, x, training=True):
    del training
    return predict_on_batch(self._variables, x)
    
  @tf.function
  def forward_pass(self, batch, training=True):
    del training
    loss, predictions = mnist_forward_pass(self._variables, batch)
    num_exmaples = tf.shape(batch['x'])[0]
    return tff.learning.models.BatchOutput(
        loss=loss, predictions=predictions, num_examples=num_exmaples)

  @tf.function
  def report_local_unfinalized_metrics(
      self) -> collections.OrderedDict[str, list[tf.Tensor]]:
    """Creates an `OrderedDict` of metric names to unfinalized values."""
    return get_local_unfinalized_metrics(self._variables)

  def metric_finalizers(
      self) -> collections.OrderedDict[str, Callable[[list[tf.Tensor]], tf.Tensor]]:
    """Creates an `OrderedDict` of metric names to finalizers."""
    return get_metric_finalizers()

  @tf.function
  def reset_metrics(self):
    """Resets metrics variables to initial value."""
    for var in self.local_variables:
      var.assign(tf.zeros_like(var))

보시다시피 `tff.learning.models.VariableModel`로 정의한 추상 메서드와 속성은 변수를 도입하고 손실 및 통계를 정의한 이전 섹션의 코드 스니펫에 해당합니다.

다음은 강조할 만한 몇 가지 사항입니다.

- TFF는 런타임에 Python을 사용하지 않으므로 모델에서 사용할 모든 상태를 TensorFlow 변수로 캡처해야 합니다(코드는 모바일 기기에 배포할 수 있도록 작성되어야 합니다. 그 이유에 대한 자세한 내용은 [사용자 정의 알고리즘](custom_federated_algorithms_1.ipynb) 튜토리얼을 참조하세요).
- 일반적으로 TFF는 강력한 형식의 환경이며 모든 구성 요소에 대한 형식 서명을 결정하려고 하기 때문에 모델은 허용하는 데이터 형식(`input_spec`)을 설명해야 합니다. 모델의 입력 형식을 선언하는 것은 필수입니다.
- 기술적으로는 필요하지 않지만, 모든 TensorFlow 로직(순방향 전달, 메트릭 계산 등)을 `tf.function`로 래핑하는 것이 좋습니다. 이렇게 하면 TensorFlow가 직렬화될 수 있고 명시적인 제어 종속성이 필요하지 않습니다.


위의 내용은 Federated SGD와 같은 평가 및 알고리즘에 충분합니다. 그러나 Federated Averaging의 경우 모델이 각 배치에서 로컬로 훈련하는 방법을 지정해야 합니다. Federated Averaging 알고리즘을 빌드할 때 로컬 옵티마이저를 지정합니다.

### 새 모델로 페더레이션 훈련 시뮬레이션하기

위의 모든 사항이 준빈되면, 프로세스의 나머지 부분은 이미 본 것과 같이 보입니다. 모델 생성자를 새 모델 클래스의 생성자로 교체하고 생성한 반복 프로세스에서 두 개의 페더레이션 계산을 사용하여 훈련 라운드를 순환합니다.

In [ ]:
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    MnistModel,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02))

In [ ]:
train_state = training_process.initialize()

In [ ]:
result = training_process.next(train_state, federated_train_data)
train_state = result.state
metrics = result.metrics
print('round  1, metrics={}'.format(metrics))

round  1, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('num_examples', 4860.0), ('loss', 3.119374), ('accuracy', 0.12345679)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])


In [ ]:
for round_num in range(2, 11):
  result = training_process.next(train_state, federated_train_data)
  train_state = result.state
  metrics = result.metrics
  print('round {:2d}, metrics={}'.format(round_num, metrics))

round  2, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('num_examples', 4860.0), ('loss', 2.98514), ('accuracy', 0.14012346)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  3, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('num_examples', 4860.0), ('loss', 2.8617127), ('accuracy', 0.1590535)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  4, metrics=OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('num_examples', 4860.0), ('loss', 2.740137), ('accuracy', 0.17860082)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
round  5, metrics=OrderedDict([('distributor', ()), ('client_work', 

TensorBoard 내에서 이들 메트릭을 보려면, 위의 "TensorBoard에서 모델 메트릭 표시하기"에 나열된 단계를 참조하세요.

## 평가

지금까지의 모든 실험은 페더레이션 훈련 메트릭(라운드의 모든 클라이언트에 걸쳐 훈련된 모든 데이터 배치에 대한 평균 메트릭)만 제시했습니다. 이는 특히 단순성을 위해 각 라운드에서 같은 클라이언트 세트를 사용했기 때문에 과대적합에 대한 일반적인 우려가 있지만, Federated Averaging 알고리즘에 특정한 훈련 메트릭에는 과대적합이라는 추가 개념이 있습니다. 이것은 각 클라이언트가 단일 데이터 배치를 가지고 있다고 상상하고 많은 반복(epoch) 동안 해당 배치에 대해 훈련하는 경우 가장 쉽게 확인할 수 있습니다. 이 경우 로컬 모델은 해당 배치 하나에 빠르게 정확히 맞으므로 평균적인 로컬 정확성 메트릭은 1.0에 접근합니다. 따라서 이들 훈련 메트릭은 훈련이 진행되고 있다는 신호로 간주될 수 있지만, 그 이상은 아닙니다.

페더레이션 데이터에 대한 평가를 수행하려면, <code>tff.learning.build_federated_evaluation</code> 함수를 사용하고 모델 생성자에 인수로 전달하는, 이 용도로 설계된 또 다른 <em>페더레이션 계산</em>을 구성할 수 있습니다. `MnistTrainableModel`를 사용했던 Federated Averaging과는 달리, `MnistMode`을 전달하면 충분합니다. 평가는 경사 하강을 수행하지 않으며 옵티마이저를 구성할 필요가 없습니다.

실험과 연구를 위해 중앙 집중식 테스트 데이터세트를 사용할 수 있는 경우, [텍스트 생성을 위한 페더레이션 학습](federated_learning_for_text_generation.ipynb)은 다른 평가 옵션을 보여줍니다. 페더레이션 학습에서 훈련된 가중치를 가져와 표준 Keras 모델에 적용한 다음 중앙 집중식 데이터세트에서 `tf.keras.models.Model.evaluate()`를 호출하면 됩니다.

In [ ]:
evaluation_process = tff.learning.algorithms.build_fed_eval(MnistModel)

다음과 같이 평가 함수의 추상 형식 서명을 검사할 수 있습니다.

In [ ]:
print(evaluation_process.next.type_signature.formatted_representation())

(<
  state=<
    global_model_weights=<
      trainable=<
        float32[784,10],
        float32[10]
      >,
      non_trainable=<>
    >,
    distributor=<>,
    client_work=<
      <>,
      <
        num_examples=<
          float32
        >,
        loss=<
          float32,
          float32
        >,
        accuracy=<
          float32,
          float32
        >
      >
    >,
    aggregator=<
      value_sum_process=<>,
      weight_sum_process=<>
    >,
    finalizer=<>
  >@SERVER,
  client_data={<
    x=float32[?,784],
    y=int32[?,1]
  >*}@CLIENTS
> -> <
  state=<
    global_model_weights=<
      trainable=<
        float32[784,10],
        float32[10]
      >,
      non_trainable=<>
    >,
    distributor=<>,
    client_work=<
      <>,
      <
        num_examples=<
          float32
        >,
        loss=<
          float32,
          float32
        >,
        accuracy=<
          float32,
          float32
        >
      >
    >,
    aggregator=<
      value_

평가 프로세스는 `tff.lenaring.templates.LearningProcess` 객체라는 점에 유의해야 합니다. 이 객체에는 상태를 생성하는 `initialize` 메서드가 있지만 처음에는 학습되지 않은 모델이 포함됩니다. `set_model_weights` 메서드를 사용할 때에는 평가할 학습 상태의 가중치를 삽입해야 합니다.

In [ ]:
evaluation_state = evaluation_process.initialize()
model_weights = training_process.get_model_weights(train_state)
evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)

이제 평가할 모델 가중치가 포함된 평가 상태를 사용하면 학습에서와 마찬가지로 프로세스에서 `next` 메서드를 호출하고 평가 데이터세트를 사용하여 평가 메트릭을 계산할 수 있습니다.

그러면 `tff.learning.templates.LearingProcessOutput` 인스턴스가 다시 반환됩니다.

In [ ]:
evaluation_output = evaluation_process.next(evaluation_state, federated_train_data)

평가 결과는 다음과 같습니다. 위의 마지막 훈련 라운드에서 보고된 것보다 수치가 약간 더 좋아 보입니다. 일반적으로, 반복 훈련 프로세스에서 보고된 훈련 메트릭은 일반적으로 훈련 라운드 시작 시 모델의 성능을 반영하므로 평가 메트릭은 항상 한 단계 앞서 있습니다.

In [ ]:
str(evaluation_output.metrics)

"OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('num_examples', 4860.0), ('loss', 1.6654209), ('accuracy', 0.3621399)])), ('total_rounds_metrics', OrderedDict([('num_examples', 4860.0), ('loss', 1.6654209), ('accuracy', 0.3621399)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])"

이제 페더레이션 데이터의 테스트 샘플을 컴파일하고 테스트 데이터에 대한 평가를 다시 실행해 보겠습니다. 데이터는 실제 사용자의 같은 샘플에서 제공되지만, 별개의 보류된 데이터세트에서 제공됩니다.

In [ ]:
federated_test_data = make_federated_data(emnist_test, sample_clients)

len(federated_test_data), federated_test_data[0]

(10,
 <_PrefetchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))])>)

In [ ]:
evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)

In [ ]:
str(evaluation_output.metrics)

"OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('num_examples', 580.0), ('loss', 1.7750846), ('accuracy', 0.33620688)])), ('total_rounds_metrics', OrderedDict([('num_examples', 580.0), ('loss', 1.7750846), ('accuracy', 0.33620688)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])"

이것으로 튜토리얼을 마칩니다. 매개변수(예: 배치 크기, 사용자 수, epoch, 학습률 등)를 사용하여 위의 코드를 수정하여 각 라운드에서 무작위 사용자 샘플에서 훈련을 시뮬레이션하고 기타 튜토리얼을 탐색해 보는 것이 좋습니다.